In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from os import listdir
from relation_matrix_all import process

## mutagenesis data

In [6]:
relation_matrix = pd.read_csv("mutagenesis_relationMatrix.csv", index_col="index")
relation_matrix
#================= config here =================
"""
I constructed the ground_truth following the rules:
    1. the arrows in dataset website: non-master table -> master table
    2. treat master_col as prime key (should contains no nan values, no duplicates)
"""

# set of (col_x, col_y), foreign key relations
# means: col_x is the foreign key in table a, col_y is the primary key in table b.
# col_x ---> col_y


groud_truth = set([ ("ATOM.CSV_molecule_id", "MOLECULE.CSV_molecule_id")
                  , ("BOND.CSV_atom1_id", "ATOM.CSV_atom_id")
                  , ("BOND.CSV_atom2_id", "ATOM.CSV_atom_id")
                  ])

In [7]:
# relation matrix ===> master-foreign relationships
import re
data_path = "../test_data/mutagenesis/"

index = relation_matrix.keys()
relation_matrix = relation_matrix.fillna(0)
threshold = 0.2

result = set()
print ("found matched col pairs: (foreign key ==> primary key \n")
for master_col_name in index:
    for other_col_name in index:
        val = relation_matrix[master_col_name][other_col_name]
        if (val > threshold):
            # one hard-coded rule appied here: master_col should contains no nan values, no duplicates
            split = re.split(".CSV_", master_col_name)
            filename = split[0].lower() + ".csv"
            data = pd.read_csv(data_path + filename)
            if (data[split[1]].duplicated().any() > 0 or data[split[1]].isnull().any()):
                break
                
            result.add((master_col_name,other_col_name))
            print ("{} <== {} : {}".format(master_col_name,other_col_name, val))


found matched col pairs: (master column (x) <== non-master column (y)) 

MOLECULE.CSV_molecule_id <== ATOM.CSV_molecule_id : 1.0


In [17]:
intersection = result.intersection(groud_truth)
precision = len(intersection)/float(len(result))
recall = len(intersection)/float(len(groud_truth))
print ("precision is： {}, recall is: {}".format(precision, recall))

precision is： 1.0, recall is: 0.3333333333333333


## finacial data

In [ ]:
relation_matrix = pd.read_csv("financial_relationMatrix.csv", index_col="index")
relation_matrix

#================= config here =================
"""
I constructed the ground_truth following the rules:
    1. the arrows in dataset website: non-master table -> master table
    2. treat master_col as prime key (should contains no nan values, no duplicates)
"""

# set of (master_col, non-master col)
groud_truth = set([("ACCOUNT.CSV_account_id", "LOAN.CSV_account_id")
                  , ("ACCOUNT.CSV_account_id", "ORDER.CSV_account_id")
                  , ("ACCOUNT.CSV_account_id", "TRANS.CSV_account_id")
                  , ("ACCOUNT.CSV_account_id", "DISP.CSV_account_id")
                  , ("DISP.CSV_disp_id", "CARD.CSV_disp_id")
                  , ("CLIENT.CSV_client_id", "DISP.CSV_client_id")
                  , ("DISRTICT.CSV_district_id", "ACCOUNT.CSV_district_id")
                  , ("DISRTICT.CSV_district_id", "CLIENT.CSV_district_id")
                  ])